# Posture detection using tensorflow判斷人體姿態<br>**Bonus**: Step 7


Although our code for Step 6 can achieve the basic function of counting squats, due to the heavy computation resources needed by Movenet, the operation is too slow, resulting in our counter running less smoothly and responsively.<br>
雖然我們Step6的代碼能夠實現深蹲計數的基本功能，但由於Movenet需要大量的計算資源，運行太過緩慢，使得我們的計數器運行不夠流暢和靈敏。<br>
<br>
The Coral accelerator is a hardware accelerator specifically designed to speed up deep learning inference, and can be used with Raspberry Pi 4. It can greatly improve the model inference speed, making it more smooth to run Movenet on Raspberry Pi 4.<br>
Coral accelerator是一種專門設計用於加速深度學習推理的硬體加速器，可以與Raspberry Pi 4一起使用。Coral accelerator可以大大提高模型推理速度，從而使得在Raspberry Pi 4上運行Movenet變得更加流暢。<br>

##### <hr>First, we import the necessary libraries and redefine some of the functions used before.<br>首先，我們導入所需要的庫，並重新定義一些之前的函數

In [2]:
from IPython.display import display, Image
import IPython
import ipywidgets as widgets
import threading
import cv2
import numpy as np

def putText(frame, text, text_color = (0, 255, 0), text_position = (50, 50)):
    # Define the text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_scale = 0.65
    text_thickness = 2

    # Add text annotation on the frame
    cv2.putText(frame, text, text_position, font, text_scale, text_color, text_thickness)

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

def calculate_hip_joint_angle(keypoints_with_scores, keypoint_threshold = 0.2): 
    
    # get the position of keypoints from MoveNet output
    a1y, a1x, a1s = keypoints_with_scores[KEYPOINT_DICT["left_shoulder"]]
    a2y, a2x, a2s = keypoints_with_scores[KEYPOINT_DICT["right_shoulder"]]
    b1y, b1x, b1s = keypoints_with_scores[KEYPOINT_DICT["left_hip"]]
    b2y, b2x, b2s = keypoints_with_scores[KEYPOINT_DICT["right_hip"]]
    c1y, c1x, c1s = keypoints_with_scores[KEYPOINT_DICT["left_knee"]]
    c2y, c2x, c2s = keypoints_with_scores[KEYPOINT_DICT["right_knee"]]

    # calculate angle of left and right body respectively
    angle1 = calculate_angle( (a1y, a1x), (b1y, b1x), (c1y, c1x) )
    angle2 = calculate_angle( (a2y, a2x), (b2y, b2x), (c2y, c2x) )

    # if confident score of keypoints are all above threshold, return the midpoint of two angle
    # otherwise, return None
    if (a1s>keypoint_threshold)*(b1s>keypoint_threshold)*(c1s>keypoint_threshold):
        return (angle1 + angle2) / 2
    else:
        return None



def calculate_knee_joint_angle(keypoints_with_scores, keypoint_threshold = 0.2): 

    # get the position of keypoints from MoveNet output
    a1y, a1x, a1s = keypoints_with_scores[KEYPOINT_DICT["left_hip"]]
    a2y, a2x, a2s = keypoints_with_scores[KEYPOINT_DICT["right_hip"]]
    b1y, b1x, b1s = keypoints_with_scores[KEYPOINT_DICT["left_knee"]]
    b2y, b2x, b2s = keypoints_with_scores[KEYPOINT_DICT["right_knee"]]
    c1y, c1x, c1s = keypoints_with_scores[KEYPOINT_DICT["left_ankle"]]
    c2y, c2x, c2s = keypoints_with_scores[KEYPOINT_DICT["right_ankle"]]

    # calculate angle of left and right body respectively
    angle1 = calculate_angle( (a1y, a1x), (b1y, b1x), (c1y, c1x) )
    angle2 = calculate_angle( (a2y, a2x), (b2y, b2x), (c2y, c2x) )

    # if confident score of keypoints are all above threshold, return the midpoint of two angle
    # otherwise, return None
    if (a1s>keypoint_threshold)*(b1s>keypoint_threshold)*(c1s>keypoint_threshold):
        return (angle1 + angle2) / 2
    else:
        return None


stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Display function
# ================
import time

def view( button, source=0, rotate=0, process=(lambda x:x)):
    """
    Args:
    button: An IPywidget button to control the function. The display stops if button.value == True.
    source: An optional integer or filename to specify the source of the video stream.
    rotate: optional integer. Set to 0 by default (not rotated). Set to 1 will rotate by 90 deg.
    process: optional function that processes each frame through process(frame) before displaying.
        Set to identity function by default.
    """
    global time_process, start_time
    time_process = []
    
    display_handle=display("Please wait...", display_id=True)
    
    cap = cv2.VideoCapture(source)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

    start_time = time.time()
    while True:
        start_time_process = time.time()
        ret, frame = cap.read()
        if button.value==True or not ret:
            cap.release()
            display_handle.update(None)
            button.value = False
            break

        #frame = cv2.flip(frame, 1)  # if the camera mirrors your image

        # Rotate the frame if rotate==1
        if rotate:
            frame = cv2.transpose(frame)
            frame = cv2.flip(frame, 0)
        
        frame = process(frame)
        
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(IPython.display.Image(data=frame.tobytes()))

        time_process.append(time.time() - start_time_process)
        

##### Next, let's try using Python to drive the Coral Accelerator to perform image inference.<br>接下來，讓我們試着用python驅動Coral Accelerator去執行圖片推理<br>
After importing the libraries, we defined function `get_result_from_image_with_model`. This function takes an image as input and applies a pre-trained pose detection model to extract the positions of key points on the human body in the image. With this function, we can quickly obtain MoveNet results using the Coral Accelerator by simply providing an image and an interpreter.<br>
導入相關庫之後，我們定義了`get_result_from_image_with_model`函數。這個函數需要輸入一張圖像並應用預先訓練的姿勢判斷模型，以提取圖像中人體關鍵點的位置。這個函數讓我們只需給定一張圖像和解釋器，就能夠利用Coral Accelerator快速得出MoveNet結果了。<br>

In [3]:
import PIL
from PIL import Image
from PIL import ImageDraw
from pycoral.adapters import common
from pycoral.utils.edgetpu import make_interpreter

interpreter = make_interpreter("movenet_single_pose_lightning_ptq_edgetpu.tflite")
interpreter.allocate_tensors()

_NUM_KEYPOINTS = 17

def get_result_from_image_with_model(input, interpreter, output=None):
    '''
    This function takes an input image and applies a pre-trained pose estimation model 
    to extract the position of human body keypoints in the image. 
    The TensorFlow Lite interpreter object is used to run inference on the input image
    using the pre-trained model.
    If the optional output argument is provided, the function also saves the input image
    with the detected keypoints drawn on it to the specified file path.

    Args:
    input: An input image file path or a numpy array representing an image.
    interpreter: A TensorFlow Lite interpreter object.
    output (optional): A file path to save the output image with keypoints drawn on it.

    Return:
    Array of keypoints. Each keypoint is represented as a triplet of [y, x, score].
    '''
    # load the input image using PIL
    if isinstance(input, str):
        img = PIL.Image.open(input)
    elif isinstance(input, np.ndarray):
        img = PIL.Image.fromarray(cv2.cvtColor(input, cv2.COLOR_BGR2RGB))

    # resize the image to model required size
    resized_img = img.resize(common.input_size(interpreter), PIL.Image.Resampling.LANCZOS)

    # load the resized image to interpreter
    common.set_input(interpreter, resized_img)

    # conduct the inference
    interpreter.invoke()

    # reshape and assign the inference result to variable `pose`
    keypoints_with_scores = common.output_tensor(interpreter, 0).copy().reshape(_NUM_KEYPOINTS, 3)

    # draw the keypoints and save the image (if specified `output`)
    if output:
        draw = ImageDraw.Draw(img)
        width, height = img.size
        for i in range(0, _NUM_KEYPOINTS):
            draw.ellipse(
            xy=[
                keypoints_with_scores[i][1] * width - 2, keypoints_with_scores[i][0] * height - 2,
                keypoints_with_scores[i][1] * width + 2, keypoints_with_scores[i][0] * height + 2
            ],
            fill=(255, 0, 0))
        img.save(output)
        
    return keypoints_with_scores

if __name__ == "__main__":
    keypoints_with_scores = get_result_from_image_with_model("input_image.jpeg", interpreter)
    print(type(keypoints_with_scores), keypoints_with_scores.shape, keypoints_with_scores, sep="\n\n")

<class 'numpy.ndarray'>

(17, 3)

[[0.32774833 0.589947   0.5694627 ]
 [0.3113609  0.59814066 0.5694627 ]
 [0.3113609  0.56536585 0.75382113]
 [0.31545776 0.5776564  0.36462   ]
 [0.31545776 0.50391304 0.70056206]
 [0.41378227 0.5694627  0.5694627 ]
 [0.4014917  0.43426654 0.43016967]
 [0.5121068  0.70875573 0.29907033]
 [0.5121068  0.53259104 0.75382113]
 [0.42197597 0.6964652  0.49981618]
 [0.4178791  0.6513998  0.29907033]
 [0.59814066 0.38920113 0.75382113]
 [0.6145281  0.2867798  0.70056206]
 [0.6595935  0.58175325 0.49981618]
 [0.6800778  0.41378227 0.6350124 ]
 [0.8398551  0.52030045 0.19664899]
 [0.9013079  0.33594203 0.43016967]]


Can this code produce the effect of Step6? Check the speed in the last cell.<br>
這個代碼能運行出Step6的效果嗎？在最後的單元格看看速度如何？

In [4]:
repetition = squat = squatting = 0
CONSECUTIVE_TH=3
def frame_process(frame):
    global repetition, squat, squatting
    
    # calculate and display angle
    keypoints_with_scores = get_result_from_image_with_model(frame, interpreter)
    
    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)

    # check if confidence is qualified, if yes, proceed to judgement
    if hip_angle is not None and knee_angle is not None:
        # if capture continuous CONSECUTIVE_TH angles in squatting, change squat to 1
        # if capture continuous CONSECUTIVE_TH angles in standing, change squat to 0 and repetition += 1
        if hip_angle<125 and knee_angle<105:
            if squat == 0:
                squatting += 1
            else:
                squatting = CONSECUTIVE_TH
        elif hip_angle>145 and knee_angle>125:
            if squat == 1:
                squatting -= 1
            else:
                squatting = 0
        if squatting == 0 and squat == 1:
            repetition += 1
            squat = 0
        elif squatting == CONSECUTIVE_TH and squat == 0:
            squat = 1
        text = f"Reps: {repetition}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    else:
        text = f"Reps: {repetition}  Knee: ?  Hip: ?"
    if squat:
        text_color = (0,255,0)
    else:
        text_color = (128,192,64)
        
    putText(frame, text, text_color)
    return frame


# Run
# ================
if __name__ == "__main__":
    display(stopButton)
    thread = threading.Thread(target=view, args=(stopButton, 0, 0, frame_process))
    thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (961) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [5]:
average_time = sum(time_process) / len(time_process)

print(f"Average process time per frame is: { average_time }")
print(f"Average frame rate is: { 1 / average_time }")

Average process time per frame is: 0.033530580030905234
Average frame rate is: 29.823522261717425


### Squat: Two-player battle version.<br>深蹲：雙人對戰版

In [7]:


repetition1 = squat1 = squatting1 = repetition2 = squat2 = squatting2 = 0
CONSECUTIVE_TH=3
def frame_process(frame):    
    global repetition1, squat1, squatting1, repetition2, squat2, squatting2
    
    frame_height, frame_width = frame.shape[:2]
    frame1 = (frame.transpose((1, 0, 2))[:frame_width//2]).transpose((1, 0, 2))
    frame2 = (frame.transpose((1, 0, 2))[frame_width//2:]).transpose((1, 0, 2))
    
    # calculate and display angle
    keypoints_with_scores1 = get_result_from_image_with_model(frame1, interpreter)
    keypoints_with_scores2 = get_result_from_image_with_model(frame2, interpreter)
    
    hip_angle1 = calculate_hip_joint_angle(keypoints_with_scores1)
    knee_angle1 = calculate_knee_joint_angle(keypoints_with_scores1)
    hip_angle2 = calculate_hip_joint_angle(keypoints_with_scores2)
    knee_angle2 = calculate_knee_joint_angle(keypoints_with_scores2)

    # check if confidence is qualified, if yes, proceed to judgement
    if hip_angle1 is not None and knee_angle1 is not None:
        # if capture continuous CONSECUTIVE_TH angles in squatting, change squat to 1
        # if capture continuous CONSECUTIVE_TH angles in standing, change squat to 0 and repetition += 1
        if hip_angle1<125 and knee_angle1<105:
            if squat1 == 0:
                squatting1 += 1
            else:
                squatting1 = CONSECUTIVE_TH
        elif hip_angle1>145 and knee_angle1>125:
            if squat1 == 1:
                squatting1 -= 1
            else:
                squatting1 = 0
        if squatting1 == 0 and squat1 == 1:
            repetition1 += 1
            squat1 = 0
        elif squatting1 == CONSECUTIVE_TH and squat1 == 0:
            squat1 = 1
        if squat1:
            text_color1 = (0,255,0)
            text1 = f"=  Reps: {repetition1}   "
        else:
            text_color1 = (64,240,160)
            text1 = f"+  Reps: {repetition1}   "
    else:
        # +=1 or -=1 the value of squatting based on current status, clamped within [0, CONSECUTIVE_TH]
        squatting1 = min(max((squatting1-1 if squat1 == 0 else squatting1+1), 0), CONSECUTIVE_TH)
        text1 = f"?  Reps: {repetition1}   "
        text_color1 = (128,128,128)
    frame1 = cv2.resize(frame1, (int(200/frame_height*frame_width), 400))
    putText(frame1, text1, text_color1)

    if hip_angle2 is not None and knee_angle2 is not None:
        # if capture continuous CONSECUTIVE_TH angles in squatting, change squat to 1
        # if capture continuous CONSECUTIVE_TH angles in standing, change squat to 0 and repetition += 1
        if hip_angle2<125 and knee_angle2<105:
            if squat2 == 0:
                squatting2 += 1
            else:
                squatting2 = CONSECUTIVE_TH
        elif hip_angle2>145 and knee_angle2>125:
            if squat2 == 1:
                squatting2 -= 1
            else:
                squatting2 = 0
        if squatting2 == 0 and squat2 == 1:
            repetition2 += 1
            squat2 = 0
        elif squatting2 == CONSECUTIVE_TH and squat2 == 0:
            squat2 = 1
        if squat2:
            text_color2 = (0,255,0)
            text2 = f"=  Reps: {repetition2}   "
        else:
            text_color2 = (64,240,160)
            text2 = f"+  Reps: {repetition2}   "
    else:
        # +=1 or -=1 the value of squatting based on current status, clamped within [0, CONSECUTIVE_TH]
        squatting2 = min(max((squatting2-1 if squat2 == 0 else squatting2+1), 0), CONSECUTIVE_TH)
        text2 = f"?  Reps: {repetition2} "
        text_color2 = (128,128,128)
    
    frame2 = cv2.resize(frame2, (int(200/frame_height*frame_width), 400))
    putText(frame2, text2, text_color2)

    frame = np.concatenate((frame1, np.zeros((400, 10 ,3 )) , frame2  ) , axis=1)
    
    putText(frame, f"Time: {int(time.time() - start_time)}s  FPS: "
        f"{int(1/(sum(time_process[-5:])/len(time_process[-5:]))) if time_process else 'N/A'}",
        (160,224,160) , text_position=(50, 20))
    
    return frame


    
# Run
# ================
if __name__ == "__main__":
    display(stopButton)
    thread = threading.Thread(target=view, args=(stopButton, 0, 0, frame_process))
    thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None